# Projeto de Sinais e Sistemas para Computação (SSC)
## I - Pré-processamento de Sinais Digitais

Este notebook implementa técnicas fundamentais de pré-processamento de sinais digitais, incluindo:
- Carregamento e visualização de dados
- Aplicação de filtros digitais
- Normalização e condicionamento de sinais
- Análise no domínio do tempo e frequência

### Objetivos
- Preparar os dados para extração de características
- Aplicar técnicas de filtragem adequadas
- Implementar métodos de normalização
- Visualizar os resultados do pré-processamento

## 1. Importação de Bibliotecas

In [1]:
# Bibliotecas principais
import numpy as np
import pandas as pd
from scipy import signal
from scipy.fft import fft, fftfreq
import wfdb
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('Agg')
%matplotlib inline

import sys
import time as time_module
from datetime import datetime
import os
import json
import glob

import warnings
warnings.filterwarnings('ignore')

# Importar nosso módulo de pré-processamento
sys.path.append('../src')
from preprocessing import (
    load_signal_data, 
    aplicar_filtro, 
    normalizar_sinal,
    remover_baseline_drift,
    detectar_outliers,
    verificar_qualidade_sinal,
    pipeline_preprocessamento,
    salvar_dados_processados
)

# Configurações de visualização
plt.style.use('default')
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 10

## 2. Carregamento e Visualização dos Dados

Nesta seção, carregamos os dados de sinais e realizamos uma análise exploratória inicial.

In [2]:
# Definir caminho para os dados
dados_base = "../data/raw/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/"
file_path_exemplo = dados_base + "records100/00000/00001_lr"

print("CARREGAMENTO E ANÁLISE DOS DADOS PTB-XL")
print("="*60)

# Primeiro: carregar metadados do dataset completo
print("1. Carregando metadados do dataset PTB-XL...")
try:
    metadata_ptbxl_path = dados_base + "ptbxl_database.csv"
    ptbxl_metadata = pd.read_csv(metadata_ptbxl_path)
    print(f"    Dataset carregado: {len(ptbxl_metadata)} registros total")
    
    # Filtrar registros disponíveis (100Hz)
    registros_disponíveis = ptbxl_metadata[ptbxl_metadata['filename_lr'].notna()].copy()
    print(f"    Registros com 100Hz disponíveis: {len(registros_disponíveis)}")

    # Mostrar estatísticas do dataset
    print(f"\n  ESTATÍSTICAS DO DATASET:")
    print(f"   Idade média: {registros_disponíveis['age'].mean():.1f} ± {registros_disponíveis['age'].std():.1f} anos")
    print(f"   Distribuição por sexo:")
    sex_counts = registros_disponíveis['sex'].value_counts()
    for sex, count in sex_counts.items():
        if sex == 0:
            print(f"     Mulher: {count} ({100*count/len(registros_disponíveis):.1f}%)")
        else:
            print(f"     Homem: {count} ({100*count/len(registros_disponíveis):.1f}%)")
    
except Exception as e:
    print(f"    Erro ao carregar metadados: {e}")
    print("    Processando apenas exemplo individual...")
    # Fallback: criar DataFrame mínimo para continuar
    registros_disponíveis = pd.DataFrame({
        'ecg_id': [1],
        'patient_id': [1], 
        'filename_lr': ['records100/00000/00001_lr'],
        'age': [50],
        'sex': [0]
    })

# Segundo: exemplo detalhado com um paciente
print(f"\n2. ANÁLISE DETALHADA - PACIENTE EXEMPLO (00001)")
print("="*60)

try:
    sinal_original, metadata = load_signal_data(file_path_exemplo)
    print(f"    Dados carregados com sucesso!")
    print(f"    Shape do sinal: {sinal_original.shape}")
    print(f"    Frequência de amostragem: {metadata['fs']} Hz")
    print(f"    Número de canais: {metadata['n_channels']}")
    print(f"    Derivações: {metadata['sig_name']}")
    print(f"    Duração: {metadata['n_samples']/metadata['fs']:.1f} segundos")
except Exception as e:
    print(f"            Erro ao carregar dados: {e}")
    raise

# Visualização detalhada do exemplo
fig, axes = plt.subplots(4, 3, figsize=(18, 12))
fig.suptitle('Visualização Detalhada - ECG de 12 Derivações (Paciente 00001)', fontsize=16, fontweight='bold')

time_axis = np.arange(len(sinal_original)) / metadata['fs']

for i, ax in enumerate(axes.flat):
    if i < metadata['n_channels']:
        ax.plot(time_axis, sinal_original[:, i], 'b-', linewidth=0.8)
        ax.set_title(f"{metadata['sig_name'][i]}", fontweight='bold')
        ax.set_xlabel('Tempo (s)')
        ax.set_ylabel('Amplitude (mV)')
        ax.grid(True, alpha=0.3)
        
        # Estatísticas básicas
        rms = np.sqrt(np.mean(sinal_original[:, i]**2))
        ax.text(0.02, 0.95, f'RMS: {rms:.3f}', transform=ax.transAxes, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7))
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()

# Verificar qualidade do exemplo
print("\n   ANÁLISE DE QUALIDADE DO EXEMPLO")
print("="*50)
qualidade_original = verificar_qualidade_sinal(sinal_original, metadata['fs'])

for canal, metricas in qualidade_original.items():
    derivacao = metadata['sig_name'][int(canal.split('_')[1])]
    print(f"    {derivacao}:")
    print(f"    SNR: {metricas['snr_estimado']:.1f} dB")
    print(f"    Amplitude máxima: {metricas['amplitude_maxima']:.3f} mV")
    print(f"    Saturação: {metricas['saturacao']*100:.2f}%")
    print(f"    RMS: {metricas['rms']:.3f}")

print(f"Dados disponíveis para processamento: {len(registros_disponíveis)} registros")
print("="*60)

CARREGAMENTO E ANÁLISE DOS DADOS PTB-XL
1. Carregando metadados do dataset PTB-XL...
    Erro ao carregar metadados: [Errno 2] No such file or directory: '../data/raw/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/ptbxl_database.csv'
    Processando apenas exemplo individual...

2. ANÁLISE DETALHADA - PACIENTE EXEMPLO (00001)
            Erro ao carregar dados: Erro ao carregar arquivo ../data/raw/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/records100/00000/00001_lr: [Errno 2] No such file or directory: '/home/saposopa/Saparia/Projeto_SSC/data/raw/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/records100/00000/00001_lr.hea'


ValueError: Erro ao carregar arquivo ../data/raw/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/records100/00000/00001_lr: [Errno 2] No such file or directory: '/home/saposopa/Saparia/Projeto_SSC/data/raw/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/records100/00000/00001_lr.hea'

## 2.1 Filtragem Digital

Aplicação de diferentes tipos de filtros para remover ruído e componentes indesejados do sinal se necessário.

In [ ]:
print("\n                FILTRAGEM DIGITAL - EXEMPLO")
print("="*60)

# Aplicar diferentes tipos de filtros
fs = metadata['fs']

# 1. Remover deriva da linha de base
print(" Removendo deriva da linha de base...")
sinal_sem_deriva = remover_baseline_drift(sinal_original, fs, freq_corte=0.5)

# 2. Aplicar filtro passa-banda padrão para ECG
print(" Aplicando filtro passa-banda (0.5-45 Hz)...")
sinal_filtrado = aplicar_filtro(sinal_sem_deriva, fs, tipo='bandpass', 
                               frequencias=(0.5, 45.0), ordem=4)

# 3. Detectar outliers
print(" Detectando outliers...")
outliers = detectar_outliers(sinal_filtrado, threshold=3.0)
num_outliers = np.sum(outliers)
print(f"    Outliers detectados: {num_outliers} de {sinal_filtrado.size} pontos ({100*num_outliers/sinal_filtrado.size:.3f}%)")

# Visualização comparativa - Antes e Depois da Filtragem
derivacao_exemplo = 1  # Lead II para exemplo

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Comparação: Antes e Depois da Filtragem (Lead II)', fontsize=16, fontweight='bold')

time_axis_filtro = np.arange(len(sinal_original)) / fs

# Sinal original
axes[0,0].plot(time_axis_filtro, sinal_original[:, derivacao_exemplo], 'b-', linewidth=0.8)
axes[0,0].set_title('Sinal Original')
axes[0,0].set_xlabel('Tempo (s)')
axes[0,0].set_ylabel('Amplitude (mV)')
axes[0,0].grid(True, alpha=0.3)

# Sinal filtrado
axes[0,1].plot(time_axis_filtro, sinal_filtrado[:, derivacao_exemplo], 'g-', linewidth=0.8)
axes[0,1].set_title('Sinal Filtrado')
axes[0,1].set_xlabel('Tempo (s)')
axes[0,1].set_ylabel('Amplitude (mV)')
axes[0,1].grid(True, alpha=0.3)

# Análise espectral - Original
freqs_orig = fftfreq(len(sinal_original), 1/fs)
fft_orig = np.abs(fft(sinal_original[:, derivacao_exemplo]))
mask_orig = freqs_orig > 0
axes[1,0].semilogy(freqs_orig[mask_orig], fft_orig[mask_orig], 'b-', linewidth=0.8)
axes[1,0].set_title('Espectro - Original')
axes[1,0].set_xlabel('Frequência (Hz)')
axes[1,0].set_ylabel('Amplitude')
axes[1,0].set_xlim(0, 100)
axes[1,0].grid(True, alpha=0.3)

# Análise espectral - Filtrado
fft_filt = np.abs(fft(sinal_filtrado[:, derivacao_exemplo]))
axes[1,1].semilogy(freqs_orig[mask_orig], fft_filt[mask_orig], 'g-', linewidth=0.8)
axes[1,1].set_title('Espectro - Filtrado')
axes[1,1].set_xlabel('Frequência (Hz)')
axes[1,1].set_ylabel('Amplitude')
axes[1,1].set_xlim(0, 100)
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Comparar qualidade antes e depois
print("\n   COMPARAÇÃO DE QUALIDADE: ANTES vs DEPOIS DA FILTRAGEM")
print("="*60)
qualidade_filtrado = verificar_qualidade_sinal(sinal_filtrado, fs)

print(f"    {'Derivação':<8} {' SNR Original':<12} {'   SNR Filtrado':<12} {'   Melhoria':<10}")
print("    " + "-" * 50)
for i in range(metadata['n_channels']):
    derivacao = metadata['sig_name'][i]
    snr_orig = qualidade_original[f'canal_{i}']['snr_estimado']
    snr_filt = qualidade_filtrado[f'canal_{i}']['snr_estimado']
    melhoria = snr_filt - snr_orig
    print(f"    {derivacao:<8}      {snr_orig:<12.1f}    {snr_filt:<12.1f} {melhoria:+.1f} dB")
print("="*60)

## 2.2 Normalização e Condicionamento do Sinal

Aplicação de técnicas de normalização para padronizar os sinais.

In [ ]:
print("\n            NORMALIZAÇÃO E CONDICIONAMENTO DO SINAL")
print("="*60)

# Aplicar diferentes métodos de normalização
print(" Aplicando normalização Z-score, Min-Max e Robust...")
sinal_zscore = normalizar_sinal(sinal_filtrado, metodo='zscore')

sinal_minmax = normalizar_sinal(sinal_filtrado, metodo='minmax')

sinal_robust = normalizar_sinal(sinal_filtrado, metodo='robust')

# Visualização comparativa dos métodos de normalização
derivacao_exemplo = 1  # Lead II

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle(f'Comparação de Métodos de Normalização (Lead {metadata["sig_name"][derivacao_exemplo]})', 
             fontsize=16, fontweight='bold')

time_axis_norm = np.arange(len(sinal_filtrado)) / fs

# Sinal filtrado (antes da normalização)
axes[0,0].plot(time_axis_norm, sinal_filtrado[:, derivacao_exemplo], 'b-', linewidth=0.8)
axes[0,0].set_title('Sinal Filtrado (Original)')
axes[0,0].set_xlabel('Tempo (s)')
axes[0,0].set_ylabel('Amplitude (mV)')
axes[0,0].grid(True, alpha=0.3)

# Z-score
axes[0,1].plot(time_axis_norm, sinal_zscore[:, derivacao_exemplo], 'r-', linewidth=0.8)
axes[0,1].set_title('Normalização Z-score')
axes[0,1].set_xlabel('Tempo (s)')
axes[0,1].set_ylabel('Amplitude (padronizada)')
axes[0,1].grid(True, alpha=0.3)

# Min-Max
axes[1,0].plot(time_axis_norm, sinal_minmax[:, derivacao_exemplo], 'g-', linewidth=0.8)
axes[1,0].set_title('Normalização Min-Max')
axes[1,0].set_xlabel('Tempo (s)')
axes[1,0].set_ylabel('Amplitude (0-1)')
axes[1,0].grid(True, alpha=0.3)

# Robust
axes[1,1].plot(time_axis_norm, sinal_robust[:, derivacao_exemplo], 'm-', linewidth=0.8)
axes[1,1].set_title('Normalização Robust')
axes[1,1].set_xlabel('Tempo (s)')
axes[1,1].set_ylabel('Amplitude (robust)')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Análise estatística dos métodos de normalização
print("\n     ANÁLISE ESTATÍSTICA DOS MÉTODOS DE NORMALIZAÇÃO")
print("="*60)

metodos = {
    'Original': sinal_filtrado,
    'Z-score': sinal_zscore,
    'Min-Max': sinal_minmax,
    'Robust': sinal_robust
}

print(f"{'Método':<12} {'Média':<10} {'Std':<10} {'Min':<10} {'Max':<10}")
print("-" * 55)

for nome, sinal in metodos.items():
    dados_canal = sinal[:, derivacao_exemplo]
    media = np.mean(dados_canal)
    std = np.std(dados_canal)
    minimo = np.min(dados_canal)
    maximo = np.max(dados_canal)
    print(f"{nome:<12} {media:<10.3f} {std:<10.3f} {minimo:<10.3f} {maximo:<10.3f}")

# Histogramas comparativos
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle(f' Distribuição dos Valores - Lead {metadata["sig_name"][derivacao_exemplo]}', 
             fontsize=16, fontweight='bold')

axes[0,0].hist(sinal_filtrado[:, derivacao_exemplo], bins=50, alpha=0.7, color='blue', edgecolor='black')
axes[0,0].set_title('Original')
axes[0,0].set_ylabel('Frequência')

axes[0,1].hist(sinal_zscore[:, derivacao_exemplo], bins=50, alpha=0.7, color='red', edgecolor='black')
axes[0,1].set_title('Z-score')

axes[1,0].hist(sinal_minmax[:, derivacao_exemplo], bins=50, alpha=0.7, color='green', edgecolor='black')
axes[1,0].set_title('Min-Max')
axes[1,0].set_ylabel('Frequência')
axes[1,0].set_xlabel('Amplitude')

axes[1,1].hist(sinal_robust[:, derivacao_exemplo], bins=50, alpha=0.7, color='magenta', edgecolor='black')
axes[1,1].set_title('Robust')
axes[1,1].set_xlabel('Amplitude')

plt.tight_layout()
plt.show()
print("="*60)

## 3. Pipeline Completo e Comparação Final

Nesta seção final, aplicamos o pipeline automatizado que integra todas as etapas de pré-processamento desenvolvidas anteriormente. O pipeline garante:

- **Reprodutibilidade**: Mesma sequência de processamento para todos os sinais
- **Consistência**: Parâmetros otimizados aplicados uniformemente  
- **Eficiência**: Processamento automatizado sem intervenção manual
- **Qualidade**: Verificação automática dos resultados

O pipeline aplica na ordem otimizada:
1. Carregamento dos dados WFDB
2. Remoção de deriva da linha de base
3. Filtragem passa-banda (0.5-45 Hz)
4. Normalização Z-score
5. Verificação de qualidade final

In [ ]:
print("\n           PIPELINE COMPLETO DE PRÉ-PROCESSAMENTO")
print("="*60)

# Demonstração com um exemplo
print("     Executando pipeline automatizado (exemplo)")
sinal_processado, metadata_final = pipeline_preprocessamento(
    file_path_exemplo,
    aplicar_filtro_flag=True,
    normalizar_flag=True,
    remover_deriva=True
)

# Comparação final: Original vs Processado (exemplo)
fig, axes = plt.subplots(3, 4, figsize=(20, 15))
fig.suptitle('  COMPARAÇÃO FINAL: Original vs Processado (Todas as Derivações)', 
             fontsize=16, fontweight='bold')

time_axis_pipeline = np.arange(len(sinal_original)) / fs

for i in range(metadata['n_channels']):
    row = i // 4
    col = i % 4
    
    # Sinal original
    axes[row, col].plot(time_axis_pipeline, sinal_original[:, i], 'b-', linewidth=0.8, alpha=0.7, label='Original')
    # Sinal processado
    axes[row, col].plot(time_axis_pipeline, sinal_processado[:, i], 'r-', linewidth=0.8, label='Processado')
    
    axes[row, col].set_title(f"{metadata['sig_name'][i]}", fontweight='bold')
    axes[row, col].set_xlabel('Tempo (s)')
    axes[row, col].set_ylabel('Amplitude')
    axes[row, col].grid(True, alpha=0.3)
    axes[row, col].legend()

plt.tight_layout()
plt.show()

# Relatório final de qualidade do exemplo
print("\n               RELATÓRIO FINAL DE QUALIDADE (EXEMPLO)")
print("="*70)
qualidade_final = metadata_final['qualidade']

print(f" {'Derivação':<8}    {'SNR (dB)':<10}    {'Amplitude Max':<15}   {'Saturação (%)':<15}   {'Status':<10}")
print("-" * 70)

for i in range(metadata['n_channels']):
    derivacao = metadata['sig_name'][i]
    canal_key = f'canal_{i}'
    snr = qualidade_final[canal_key]['snr_estimado']
    amp_max = qualidade_final[canal_key]['amplitude_maxima']
    saturacao = qualidade_final[canal_key]['saturacao'] * 100
    
    # Determinar status
    if snr >= 20 and saturacao < 1:
        status = "Ótimo"
    elif snr >= 15 and saturacao < 2:
        status = "Bom"
    else:
        status = "Atenção"
    
    print(f"  {derivacao:<8}    {snr:<10.1f}       {amp_max:<15.3f}  {saturacao:<15.2f}{status:<10}")

# Salvar dados do exemplo
print(f"\n  SALVANDO DADOS DO EXEMPLO")
print("="*50)

arquivos_salvos = salvar_dados_processados(
    sinal_processado, 
    metadata_final, 
    ecg_id=1,
    output_dir="../data/processed"
)

print(f"    Arquivos gerados:")
for i, arquivo in enumerate(arquivos_salvos, 1):
    nome_arquivo = arquivo.split('/')[-1]
    print(f"   {i}. {nome_arquivo}")
print("="*60)

## 3.1 Pipeline Completo em Pacote
Vamos efetivamente preprocessar todos os dados para utilizá-los depois

In [ ]:
# PROCESSAMENTO EM LOTE DE TODOS OS REGISTROS
print(f"    PROCESSAMENTO EM LOTE - TODOS OS REGISTROS PTB-XL")
print("="*70)

# Decidir quantos registros processar
registros_para_processar = registros_disponíveis  # REMOVA .head(100) para processar TODOS
print(f"    Processando {len(registros_para_processar)} registros")

# Dicionário para armazenar resultados do lote
resultados_lote = {
    'ecg_id': [],
    'patient_id': [],
    'age': [],
    'sex': [],
    'status': [],
    'snr_medio_original': [],
    'snr_medio_final': [],
    'melhoria_snr': [],
    'outliers_percent': [],
    'canais_com_boa_qualidade': [],
    'tempo_processamento': [],
    'pasta_destino': [],
    'arquivo_sinal': [],
    'arquivo_metadata': []
}

# Contadores para estatísticas
total_registros = len(registros_para_processar)
sucessos = 0
erros = 0
start_time_total = time_module.time()

# CONTADOR DE PASTAS CRIADAS
pastas_criadas = set()

# Loop através de todos os registros
for idx, (_, row) in enumerate(registros_para_processar.iterrows()):
    ecg_id = None
    patient_id = None  
    age = 0
    sex = 2  # Valor padrão para sexo desconhecido
    
    try:
        # Progresso a cada 100 registros
        if idx % 100 == 0 or idx == total_registros - 1:
            progresso = (idx + 1) / total_registros * 100
            tempo_decorrido = time_module.time() - start_time_total
            if idx > 0:
                tempo_estimado = (tempo_decorrido / (idx + 1)) * total_registros
                print(f"\n  Progresso: {idx+1}/{total_registros} ({progresso:.1f}%) - "
                      f"Sucessos: {sucessos}, Erros: {erros}")
                print(f"    Tempo decorrido: {tempo_decorrido/60:.1f}min - "
                      f"Estimativa total: {tempo_estimado/60:.1f}min")
        
        # Extrair informações do registro
        ecg_id = row['ecg_id']
        patient_id = row['patient_id']
        age = row['age'] if pd.notna(row['age']) else 0
        sex = row['sex'] if pd.notna(row['sex']) else 'unknown'
        filename = row['filename_lr']

        # Verificar se o filename está disponível
        if pd.isna(filename):
            raise ValueError("Filename não disponível")
        
        # Construir caminho completo
        file_path_completo = dados_base + filename
        
        # Medir tempo de processamento individual
        inicio_individual = time_module.time()
        
        # Aplicar pipeline de pré-processamento
        sinal_processado_lote, metadata_proc = pipeline_preprocessamento(
            file_path_completo,
            aplicar_filtro_flag=True,
            normalizar_flag=True,
            remover_deriva=True
        )
        
        # Carregar sinal original para comparação
        sinal_original_lote, metadata_original = load_signal_data(file_path_completo)
        qualidade_original_lote = verificar_qualidade_sinal(sinal_original_lote, metadata_original['fs'])
        
        # Calcular métricas
        qualidade_final_lote = metadata_proc['qualidade']
        
        snr_orig = np.mean([qualidade_original_lote[f'canal_{j}']['snr_estimado'] 
                           for j in range(metadata_proc['n_channels'])])
        snr_final = np.mean([qualidade_final_lote[f'canal_{j}']['snr_estimado'] 
                            for j in range(metadata_proc['n_channels'])])
        
        # Contar canais com boa qualidade (SNR >= 15 dB)
        canais_bons = len([i for i in range(metadata_proc['n_channels']) 
                          if qualidade_final_lote[f'canal_{i}']['snr_estimado'] >= 15])
        
        # Calcular outliers
        outliers_lote = detectar_outliers(sinal_processado_lote, threshold=3.0)
        outliers_pct = (np.sum(outliers_lote) / sinal_processado_lote.size) * 100
        
        # Salvar dados processados
        arquivos = salvar_dados_processados(
            sinal_processado_lote, 
            metadata_proc, 
            ecg_id=ecg_id,  # ECG ID como identificador
            output_dir="../data/processed"
        )
        
        # RASTREAR PASTA CRIADA
        folder_number = (ecg_id - 1) // 1000
        pasta_criada = f"records{folder_number:03d}"
        pastas_criadas.add(pasta_criada)

        tempo_individual = time_module.time() - inicio_individual
        
        # Armazenar resultados
        resultados_lote['ecg_id'].append(ecg_id)
        resultados_lote['patient_id'].append(patient_id)
        resultados_lote['age'].append(age)
        resultados_lote['sex'].append(sex)
        resultados_lote['status'].append('Sucesso')
        resultados_lote['snr_medio_original'].append(snr_orig)
        resultados_lote['snr_medio_final'].append(snr_final)
        resultados_lote['melhoria_snr'].append(snr_final - snr_orig)
        resultados_lote['outliers_percent'].append(outliers_pct)
        resultados_lote['canais_com_boa_qualidade'].append(canais_bons)
        resultados_lote['tempo_processamento'].append(tempo_individual)
        resultados_lote['pasta_destino'].append(pasta_criada)
        resultados_lote['arquivo_sinal'].append(arquivos[0])
        resultados_lote['arquivo_metadata'].append(arquivos[1])
        
        sucessos += 1
        
    # Tratamento específico para arquivo não encontrado
    except FileNotFoundError as e:
        print(f"        Arquivo não encontrado - ECG {ecg_id}: {filename}")
        # Adicionar entrada de erro
        resultados_lote['ecg_id'].append(ecg_id if ecg_id is not None else 0)
        resultados_lote['patient_id'].append(patient_id if patient_id is not None else 0)
        resultados_lote['age'].append(age)
        resultados_lote['sex'].append(sex)
        resultados_lote['status'].append('Arquivo não encontrado')
        # Adicionar zeros para outras métricas
        for key in ['snr_medio_original', 'snr_medio_final', 'melhoria_snr', 'outliers_percent', 
                   'canais_com_boa_qualidade', 'tempo_processamento']:
            resultados_lote[key].append(0)
        resultados_lote['pasta_destino'].append('N/A')
        resultados_lote['arquivo_sinal'].append('N/A')
        resultados_lote['arquivo_metadata'].append('N/A')
        erros += 1
        
    except Exception as e:
        # Tratamento robusto de erros
        print(f"    Erro no ECG {ecg_id if ecg_id is not None else 'N/A'}: {str(e)[:100]}...")
        
        # Adicionar entrada de erro com valores seguros
        resultados_lote['ecg_id'].append(ecg_id if ecg_id is not None else 0)
        resultados_lote['patient_id'].append(patient_id if patient_id is not None else 0)
        resultados_lote['age'].append(age)
        resultados_lote['sex'].append(sex)
        resultados_lote['status'].append(f'     Erro: {str(e)[:50]}')
        # Adicionar zeros para outras métricas
        for key in ['snr_medio_original', 'snr_medio_final', 'melhoria_snr', 'outliers_percent', 
                   'canais_com_boa_qualidade', 'tempo_processamento']:
            resultados_lote[key].append(0)
        resultados_lote['pasta_destino'].append('N/A')
        resultados_lote['arquivo_sinal'].append('N/A')
        resultados_lote['arquivo_metadata'].append('N/A')
        
        erros += 1

tempo_total = time_module.time() - start_time_total

# Criar DataFrame final com todos os resultados
df_resultados_lote = pd.DataFrame(resultados_lote)

print(f"\n      PROCESSAMENTO EM LOTE CONCLUÍDO!")
print("="*70)
print(f"    Tempo total: {tempo_total/60:.2f} minutos")
print(f"    Processamentos bem-sucedidos: {sucessos}/{total_registros} ({100*sucessos/total_registros:.1f}%)")
print(f"    Processamentos com erro: {erros}/{total_registros} ({100*erros/total_registros:.1f}%)")
print(f"    Total de pastas criadas: {len(pastas_criadas)}")
print(f"    Pastas: {sorted(list(pastas_criadas))}")

# Estatísticas dos processamentos bem-sucedidos
sucessos_df = df_resultados_lote[df_resultados_lote['status'] == 'Sucesso']

if len(sucessos_df) > 0:
    print(f"\n      ESTATÍSTICAS DOS SUCESSOS:")
    print(f"        SNR médio original: {sucessos_df['snr_medio_original'].mean():.1f} ± {sucessos_df['snr_medio_original'].std():.1f} dB")
    print(f"        SNR médio final: {sucessos_df['snr_medio_final'].mean():.1f} ± {sucessos_df['snr_medio_final'].std():.1f} dB")
    print(f"        Melhoria média: {sucessos_df['melhoria_snr'].mean():.1f} ± {sucessos_df['melhoria_snr'].std():.1f} dB")
    print(f"        Outliers médios: {sucessos_df['outliers_percent'].mean():.3f}% dos dados")
    print(f"        Canais com boa qualidade: {sucessos_df['canais_com_boa_qualidade'].mean():.1f}/12 canais")
    print(f"        Tempo médio por registro: {sucessos_df['tempo_processamento'].mean():.2f}s")
    
    # Estatísticas demográficas
    print(f"\n  DADOS DEMOGRÁFICOS:")
    print(f"   Idade média: {sucessos_df['age'].mean():.1f} ± {sucessos_df['age'].std():.1f} anos")
    print(f"   Distribuição por sexo:")
    sex_counts = sucessos_df['sex'].value_counts()
    for sex, count in sex_counts.items():
        if sex == 0:
            print(f"     Mulher: {count} ({100*count/len(sucessos_df):.1f}%)")
        else:
            print(f"     Homem: {count} ({100*count/len(sucessos_df):.1f}%)")

# Salvar relatórios
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Criar diretório para salvar relatórios
os.makedirs("../data/processed", exist_ok=True)

# Relatório completo
relatorio_completo_path = f"../data/processed/relatorio_processamento_completo_{timestamp}.csv"
df_resultados_lote.to_csv(relatorio_completo_path, index=False)

# Relatório apenas sucessos
relatorio_sucessos_path = f"../data/processed/relatorio_sucessos_{timestamp}.csv"
sucessos_df.to_csv(relatorio_sucessos_path, index=False)

# Relatório de erros
if erros > 0:
    erros_df = df_resultados_lote[df_resultados_lote['status'] != 'Sucesso']
    relatorio_erros_path = f"../data/processed/relatorio_erros_{timestamp}.csv"
    erros_df.to_csv(relatorio_erros_path, index=False)
    print(f"    Relatório de erros salvo: {relatorio_erros_path}")

print(f"\n  RELATÓRIOS SALVOS:")
print(f"    Relatório completo: {relatorio_completo_path}")
print(f"    Relatório sucessos: {relatorio_sucessos_path}")

print(f"\n  DADOS PRONTOS PARA O PRÓXIMO NOTEBOOK")
print(f"    Total de arquivos processados em ../data/processed/: {sucessos} arquivos")

# Mostrar amostra dos resultados
if len(sucessos_df) > 0:
    print(f"\n              AMOSTRA DOS RESULTADOS (primeiros 10 sucessos):")
    print(sucessos_df[['ecg_id', 'patient_id', 'age', 'snr_medio_final', 'canais_com_boa_qualidade']].head(10).to_string(index=False))
else:
    print(f"\n              Nenhum registro processado com sucesso.")

print("="*70)

## 4. Resumo e Conclusões

### Técnicas Implementadas:
 **Carregamento de dados WFDB (PTB-XL)** - Importação dos sinais ECG do dataset  
 **Remoção de deriva da linha de base** - Filtro passa-alta 0.5 Hz  
 **Filtragem passa-banda** - Remoção de ruídos (0.5-45 Hz)  
 **Detecção de outliers** - Identificação de artefatos usando z-score  
 **Normalização Z-score** - Padronização para algoritmos ML  
 **Verificação de qualidade** - Métricas automáticas de SNR e saturação  
 **Pipeline automatizado** - Processamento reprodutível e consistente  

### Resultados Obtidos:
- Melhoria significativa na qualidade do sinal (SNR)
- Remoção eficaz de ruídos e artefatos
- Sinais padronizados prontos para extração de características
- Dados salvos em formato adequado para os próximos notebooks

---
** Dados pré-processados salvos em [`../data/processed/`](../data/processed/)**